###  Mount Google Drive

In [1]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3


Mounted at /content/drive


### Clone Codebase

In [ ]:
! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3
! cd /content/drive/MyDrive/COMP4901B-Homework3 && git clone https://github.com/hkust-nlp/COMP4901B-LLMs.git

Cloning into 'COMP4901B-LLMs'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 232 (delta 40), reused 24 (delta 24), pack-reused 169 (from 1)
Receiving objects: 100% (232/232), 798.55 KiB | 1.87 MiB/s, done.
Resolving deltas: 100% (87/87), done.


### Download Dataset

In [4]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/setup.sh

Looking in indexes: https://download.pytorch.org/whl/cu126
Fetching 10 files: 100% 10/10 [00:00<00:00, 137.25it/s]
Fetching 6 files: 100% 6/6 [00:00<00:00, 144.61it/s]


### Q1 code evaulation for q1

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3/ && python tests/test_format_prompts.py


SANITY CHECK: format_prompts()

This script tests your implementation of the format_prompts function.
It compares your outputs with expected reference outputs.


TEST 2: Few-shot without chat template
2025-11-13 09:43:54.067356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763027034.487824    6130 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763027034.594252    6130 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763027035.283583    6130 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763027035.283628    6130 computation_placer.cc:

### Q4 Start The Tuning

#### Default

In [3]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B

GSM8K Self-Improving Training Pipeline
Run Name: first-run
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/first-run
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 128
  Batch Size Per Device: 1
  Gradient Accumulation Steps: 128 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/first-run/iteration_0/inference.jsonl

2025-11-14 11:42:24.446372: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn the

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/first-run/models/model_iter_1-merged \
    --output_dir results \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [9]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/first-run/models/model_iter_1-merged \
    --output_dir results \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/first-run/models/model_iter_1-merged
Output Directory: results
Run Name: model_iter_1-merged_20251115_060728
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/model_iter_1-merged_20251115_060728_inference.jsonl

2025-11-15 06:07:32.677374: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 06:07:32.694729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763186852.716271   

#### Increase epoch (128)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --total_batch_size 128 --run_name epoch_128

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/epoch_128/models/model_iter_1-merged \
    --output_dir results/epoch_128 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/epoch_128/models/model_iter_1-merged \
    --output_dir results/epoch_128 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Increase epoch (256)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --total_batch_size 256 --run_name epoch_256

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/epoch_256/models/model_iter_1-merged \
    --output_dir results/epoch_256 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/epoch_256/models/model_iter_1-merged \
    --output_dir results/epoch_256 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Increase queries (3000)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --n_queries 3000 --run_name queries_3000

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_3000/models/model_iter_1-merged \
    --output_dir results/queries_3000 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_3000/models/model_iter_1-merged \
    --output_dir results/queries_3000 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Decrease queries (1000)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --n_queries 1000 --run_name queries_1000

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_1000/models/model_iter_1-merged \
    --output_dir results/queries_1000 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_1000/models/model_iter_1-merged \
    --output_dir results/queries_1000 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Increase lr (4e-5)

In [1]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --learning_rate 4e-5 --run_name "lr_4e-5"

/bin/bash: line 1: cd: /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3: No such file or directory


In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_4e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_4e-5" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_4e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_4e-5" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Increase lr (1e-5)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --learning_rate 1e-5 --run_name "lr_1e-5"

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_1e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_1e-5" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_1e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_1e-5" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Increase LoRA rank (128)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --lora_r 128 --run_name rank_128

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_128/models/model_iter_1-merged \
    --output_dir results/rank_128 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_128/models/model_iter_1-merged \
    --output_dir results/rank_128 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

#### Decrease LoRA rank (32)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --lora_r 32 --run_name rank_32

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_32/models/model_iter_1-merged \
    --output_dir results/rank_32 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1 \

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_32/models/model_iter_1-merged \
    --output_dir results/rank_32 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000